In [24]:
import this

In [27]:
import requests
from bs4 import BeautifulSoup

In [18]:
url = "https://www.cian.ru/sale/flat/223309049/"
req = requests.get(url)
req

<Response [200]>

In [19]:
req.status_code

200

In [40]:
content = req.content
soup = BeautifulSoup(content)
price = soup.find_all("span", {"itemprop": "price"})[0]
price

<span content="5 279 008 ₽" itemprop="price">5 279 008 ₽</span>

In [45]:
import re
re.sub('\D', '', price.text)

'5279008'